In [10]:
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import pandas_datareader.data as dtr
import requests_cache
import datetime
expire_after = datetime.timedelta(days = 7)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)

tickers = ["LNTA.ME","MGNT.ME","PIKK.ME",
           "MTSS.ME","MGTS.ME","RTKM.ME",
           "ROSN.ME","GAZP.ME","NVTK.ME","LKOH.ME","TATN.ME","SNGS.ME","SIBN.ME",
           "PLZL.ME","NLMK.ME","CHMF.ME","POLY.ME","RUAL.ME","ALRS.ME",
           "SBER.ME","VTBR.ME","MOEX.ME","CBOM.ME","YNDX.ME","CBOM.ME","ROSB.ME",
           "HYDR.ME","RSTI.ME","FEES.ME"
          ]

df = pd.DataFrame()

for ticker in tickers:
    try:
        df_ = dtr.DataReader(ticker, data_source="yahoo" , session=session, retry_count=1)
        df_.insert(0, "ticker", ticker)
    except:
        print(f"ticker '{ticker}' failed")
        pass
    df = df.append(df_)

GAZP = df[df.ticker == "GAZP.ME"]
GAZP.head()

In [5]:
class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    up_fast = 10
    up_slow = 20
#     down_fast = 10
#     down_slow = 20
    
    def init(self):
        
        price = self.data.Close # to be fed to indicator
        self.up_fast = self.I(SMA, price, self.up_fast) # callable, callable args
        self.up_slow = self.I(SMA, price, self.up_slow)
#         self.down_fast = self.I(SMA, price, self.down_fast)
#         self.down_slow = self.I(SMA, price, self.down_slow)
        
    def next(self):
        
        if crossover(self.up_fast, self.up_slow):
            self.position.close() # close short if any
            self.buy() # buy
            
        if crossover(self.up_slow, self.up_fast):
            self.position.close() # close long if any
            self.sell() # short
            
#         if crossover(self.down_slow, self.down_fast):
#             self.sell()
            
#         if crossover(self.down_fast, self.down_slow):
#             self.position.close()
# #             self.buy()

bt = Backtest(GAZP, SmaCross, cash=10000,commission=.002, ) # exclusive_orders=True any time youre short or long 
stats = bt.optimize(up_fast=range(5, 30, 5),
                    up_slow=range(10, 70, 5),
#                     down_fast=range(5, 30, 5),
#                     down_slow=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.up_fast < param.up_slow)# and param.down_fast < param.down_slow)
stats

Start                     2015-12-17 00:00:00
End                       2020-12-14 00:00:00
Duration                   1824 days 00:00:00
Exposure Time [%]                     91.2211
Equity Final [$]                      17968.8
Equity Peak [$]                       18704.7
Return [%]                            79.6875
Buy & Hold Return [%]                 44.6773
Return (Ann.) [%]                     12.5092
Volatility (Ann.) [%]                 25.4523
Sharpe Ratio                         0.491476
Sortino Ratio                         0.88546
Calmar Ratio                         0.456775
Max. Drawdown [%]                    -27.3859
Avg. Drawdown [%]                    -5.62879
Max. Drawdown Duration      335 days 00:00:00
Avg. Drawdown Duration       56 days 00:00:00
# Trades                                   28
Win Rate [%]                          39.2857
Best Trade [%]                        41.1498
Worst Trade [%]                      -10.1933
Avg. Trade [%]                    

In [6]:
stats["_trades"]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-68,110,223,145.608194,138.000000,517.357186,0.052251,2016-06-01,2016-11-09,161 days
1,76,223,283,138.276000,149.800003,875.824232,0.083341,2016-11-09,2017-02-02,85 days
2,-76,283,349,149.500403,131.410004,1374.870353,0.121006,2017-02-02,2017-05-12,99 days
3,96,349,360,131.672824,122.400002,-890.190926,-0.070423,2017-05-12,2017-05-29,17 days
4,-97,360,399,122.155202,118.529999,351.644666,0.029677,2017-05-29,2017-07-24,56 days
5,102,399,406,118.767059,118.000000,-78.239995,-0.006459,2017-07-24,2017-08-02,9 days
6,-103,406,441,117.764000,121.889999,-424.977937,-0.035036,2017-08-02,2017-09-20,49 days
7,96,441,563,122.133779,140.300003,1743.957472,0.148740,2017-09-20,2018-03-20,181 days
8,-96,563,583,140.019403,142.449997,-233.337015,-0.017359,2018-03-20,2018-04-17,28 days
9,92,583,624,142.734897,138.050003,-431.010238,-0.032822,2018-04-17,2018-06-18,62 days


# Wrong cause doesn't check for available equity

- The below approach places orders out of thin air

In [9]:
class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    up_fast = 10
    up_slow = 20
    down_fast = 10
    down_slow = 20
    
    def init(self):
        
        price = self.data.Close # to be fed to indicator
        self.up_fast = self.I(SMA, price, self.up_fast) # callable, callable args
        self.up_slow = self.I(SMA, price, self.up_slow)
        self.down_fast = self.I(SMA, price, self.down_fast)
        self.down_slow = self.I(SMA, price, self.down_slow)
        
    def next(self):
        
        if crossover(self.up_fast, self.up_slow):
            self.position.close() # close short if any
            self.buy() # buy
            
        if crossover(self.up_slow, self.up_fast):
            self.position.close() # close long if any
            self.sell() # short
            
        if crossover(self.down_slow, self.down_fast):
            self.sell()
            
        if crossover(self.down_fast, self.down_slow):
            self.position.close()
            self.buy()

bt = Backtest(GAZP, SmaCross, cash=10000,commission=.002, ) # exclusive_orders=True -- cancels previos position 
stats = bt.run()

# stats = bt.optimize(up_fast=range(5, 30, 5),
#                     up_slow=range(10, 70, 5),
#                     down_fast=range(5, 30, 5),
#                     down_slow=range(10, 70, 5),
#                     maximize='Equity Final [$]',
#                     constraint=lambda param: param.up_fast < param.up_slow and param.down_fast < param.down_slow)
# stats

In [58]:
stats["_trades"]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,116,200,146.706000,134.880005,11.825995,0.080610,2016-06-09,2016-10-06,119 days
1,-68,114,200,146.137133,134.880005,765.484691,0.077031,2016-06-07,2016-10-06,121 days
2,79,200,221,135.149765,140.500000,422.668573,0.039587,2016-10-06,2016-11-07,32 days
3,79,221,224,140.781000,146.449997,447.850759,0.040268,2016-11-07,2016-11-10,3 days
4,79,224,287,146.742897,148.119995,108.790756,0.009384,2016-11-10,2017-02-08,90 days
5,-79,287,355,147.823755,124.470001,1844.946559,0.157984,2017-02-08,2017-05-22,103 days
6,109,355,356,124.718941,122.599998,-230.964760,-0.016990,2017-05-22,2017-05-23,1 days
7,108,356,362,122.845198,122.209999,-68.601534,-0.005171,2017-05-23,2017-05-31,8 days
8,-5,367,401,119.101316,118.349998,3.756589,0.006308,2017-06-07,2017-07-26,49 days
9,-109,362,401,121.965579,118.349998,394.098287,0.029644,2017-05-31,2017-07-26,56 days
